#**NMF**
---
**Name(s):** Thodori Kapouranis, Steven Lee

**Class:** ECE-475

**Professor:** Keene

# **Installation**

In [ ]:
!pip install numpy
!pip install scikit-surprise

     |████████████████████████████████| 11.8MB 3.9MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp36-cp36m-linux_x86_64.whl size=1618267 sha256=5c9dd3f5d42c85bcd5d23dd7c7665dadafdee35bfded93473c31ec3915cd0308
  Stored in directory: /root/.cache/pip/wheels/78/9c/3d/41b419c9d2aff5b6e2b4c0fc8d25c538202834058f9ed110d0
Successfully built scikit-surprise


# **MovieLens Dataset (100k)**



In [ ]:

from surprise import SVD
from surprise import NMF
from surprise import Dataset
from surprise.model_selection import cross_validate
from surprise.model_selection import GridSearchCV
import numpy as np
import pandas as pd

data = Dataset.load_builtin('ml-100k')
algo = NMF()
# perf = cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=3, verbose=True)
# print(perf)


Evaluating RMSE, MAE of algorithm NMF on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    0.9791  0.9666  0.9693  0.9717  0.0054  
MAE (testset)     0.7688  0.7607  0.7610  0.7635  0.0038  
Fit time          4.31    4.37    4.37    4.35    0.03    
Test time         0.37    0.20    0.35    0.30    0.07    
{'test_rmse': array([0.97914919, 0.96658925, 0.9693216 ]), 'test_mae': array([0.76883957, 0.76065397, 0.76100767]), 'fit_time': (4.308727264404297, 4.374547719955444, 4.373061895370483), 'test_time': (0.36517763137817383, 0.2010335922241211, 0.3458740711212158)}


# **Grabbing ratings and id to title mapping**

In [ ]:
def grabItems():
    file_name = (os.path.expanduser('~') +
                 '/.surprise_data/ml-100k/ml-100k/u.item')
    id_to_name = {}
    name_to_id = {}
    with io.open(file_name, 'r', encoding='ISO-8859-1') as f:
        for line in f:
            line = line.split('|')
            id_to_name[line[0]] = line[1]
            name_to_id[line[1]] = line[0]
    return id_to_name, name_to_id

id_to_name, name_to_id = grabItems()
ratings_df = pd.read_csv(data.ratings_file, delimiter='\t')
ratings_df.columns = ['userId', 'movieId', 'rating', 'timestamp']
ratings_df



,userId,movieId,rating,timestamp
0,186,302,3,891717742
1,22,377,1,878887116
2,244,51,2,880606923
3,166,346,1,886397596
4,298,474,4,884182806
...,...,...,...,...
99994,880,476,3,880175444
99995,716,204,5,879795543
99996,276,1090,1,874795795
99997,13,225,2,882399156


# **Finding optimal params**


In [ ]:
# This takes a long time to run!
param_grid = {'n_factors': [10,12,14,16,18,20], 'reg_pu':[0.01, 0.02, 0.04, 0.06], 'reg_qi':[0.01, 0.02, 0.04, 0.06]}
gs = GridSearchCV(NMF, param_grid, measures=['rmse'], cv=5)
gs.fit(data)

# best RMSE score
print(gs.best_score['rmse'])

# combination of parameters that gave the best RMSE score
print(gs.best_params['rmse'])


0.9625446224321106
{'n_factors': 16, 'reg_pu': 0.06, 'reg_qi': 0.06}


# **Training with the optimal params**

In [ ]:
trainset = data.build_full_trainset()
testset = trainset.build_anti_testset()

# Fitting the model with the optimal parameters
algo = NMF(n_factors=16)
algo.fit(trainset)

# Building predictions
pred = algo.test(testset)

# **Recommendation building function**

In [ ]:
from collections import defaultdict #data colector

def recommend(pred, id, movies, ratings, n=10):
  top_n = defaultdict(list)
  for uid, iid, true_r, est, _ in pred:
    top_n[uid].append((iid, est))
  
  for uid, user_ratings in top_n.items():
    user_ratings.sort(key = lambda x: x[1], reverse = True)
    top_n[uid] = user_ratings[: n]
  
  preds_df = pd.DataFrame([(id, pair[0],pair[1], movies[pair[0]]) for id, row in top_n.items() for pair in row],
                        columns=["User ID" ,"Movie ID","Rating prediction", "Title"])
  pred_usr = preds_df[preds_df["User ID"] == str(id)]

  hist_usr = ratings_df[ratings_df["userId"] == id].sort_values("rating", ascending = False)
  titles = [ id_to_name[str(i)] for i in hist_usr["movieId"].values ] 
  hist_usr["Titles"] = titles

  return pred_usr, hist_usr
  
pred_usr, hist_usr = recommend(pred, 365, id_to_name, ratings_df, 10)

In [ ]:
hist_usr

,userId,movieId,rating,timestamp,Titles
7106,365,301,5,891303586,In & Out (1997)
36063,365,340,5,891303536,Boogie Nights (1997)
26305,365,288,5,891303357,Scream (1996)
30054,365,1137,5,891303950,Beautiful Thing (1996)
8909,365,813,5,891303901,"Celluloid Closet, The (1995)"
44345,365,268,5,891303474,Chasing Amy (1997)
15653,365,150,5,891303950,Swingers (1996)
35173,365,100,5,891303901,Fargo (1996)
12534,365,321,5,891303536,Mother (1996)
25081,365,269,4,891303357,"Full Monty, The (1997)"


**User's history :**
---
In this users 5-star rated movies, we see a wide spread of genres:


**Drama/Comedy** : Beautiful Thing, Chasing Amy, Boogie Nights


**Drama/Thriller** : Fargo


**Drama** : Mother


**Horror** : Scream


**History** : The Celluloid Closet


**Comedy** : In & Out



In [ ]:
pred_usr

,User ID,Movie ID,Rating prediction,Title
3590,365,867,5.000000,"Whole Wide World, The (1996)"
3591,365,1368,5.000000,Mina Tannenbaum (1994)
3592,365,1467,5.000000,"Saint of Fort Washington, The (1993)"
3593,365,318,4.989719,Schindler's List (1993)
3594,365,1536,4.889612,Aiqing wansui (1994)
3595,365,1449,4.868046,Pather Panchali (1955)
3596,365,1650,4.856729,"Butcher Boy, The (1998)"
3597,365,1636,4.811802,Brothers in Trouble (1995)
3598,365,1651,4.759367,"Spanish Prisoner, The (1997)"
3599,365,1500,4.748120,Santa with Muscles (1996)


The predictions split in these categories:

**Drama/thriller** : The Spanish prisoner

**Drama** : Pather Panchali, Vive L'Amour, Saint of Fort Washington, Mina Tannenbaum,  The Whole Wide World


**Drama/History** : Schlinder's List

**Drama/comedy** : Brothers in Trouble, Butcher Boy, 


**Comedy** : Santa With Muscles




It seems that the algorithm focused on the large pile of movies that are tagged drama, and then combined them with other genres that also recieved high rating. These results make a lot of sense and are very consistent with the users history.

# **Stretch Goal 1**

In [ ]:
class myNMF:
  # We will be trying to estimate A = WH
  def __init__(self, data, latent=10, lu=0.01, li=0.01):
    self.users = len( data.all_users() )
    self.items = len( data.all_items() )
    
    # Set up the matrix that we will be trying to factorize
    # rows are users, columns are movies, entries is the rating
    self.A = np.zeros( (self.users, self.items) )
    for rating in data.all_ratings():
      self.A[rating[0], rating[1]] = rating[2]

    self.W = np.random.uniform(0,1,(self.A.shape[0], latent))
    self.H = np.random.uniform(0,1,(self.A.shape[1], latent))

    self.latent= latent
    self.lu = lu
    self.li = li

  # Referenced for update rule: 
  # https://blog.insightdatascience.com/explicit-matrix-factorization-als-sgd-and-all-that-jazz-b00e4d9b21ea
  def alt_update(self):
    # Alternating update rule
    # We must work user by user or movie by movie, because of missing values.
    for user in range(self.users):
      rated_movies = [True if i>0 else False for i in self.A[user,:] ] # User's rated movies (True/False array)
      user_ratings = self.A[user,:][rated_movies]                      # User's ratings for the above movies

      # Update the proper rows/col
      H_tmp = self.H[rated_movies, :]
      self.W[user,:] = user_ratings @ H_tmp @ np.linalg.pinv(H_tmp.T @ H_tmp + self.lu * np.eye(self.latent))
    
    for item in range(self.items):
      rated_users  = [True if i>0 else False for i in self.A[:,item]]
      user_ratings = self.A[:,item][rated_users] 
      
      W_tmp = self.W[rated_users,:]
      self.H[item,:] = user_ratings @ W_tmp @ np.linalg.inv(W_tmp.T @ W_tmp + self.li *np.eye(self.latent))

  def error(self):
    existing_predictions = np.sign(self.A) * (self.W @ self.H.T)
    return np.linalg.norm(self.A - existing_predictions, 'fro')

  def train(self, iterations, vocal=False):
    for i in range(iterations):
      self.alt_update()
      if vocal==True:
        print(f'i={i} error:{self.error()}' )

  def predict(self, user, item):
    # Find the prediction by matrix multiplying the two factored matrices
    return self.W[user,:] @ self.H[item,:].T

In [ ]:
from surprise import SVD
from surprise import NMF
from surprise import Dataset
from surprise.model_selection import cross_validate
from surprise.model_selection import GridSearchCV
from surprise.model_selection.split import train_test_split
import numpy as np
import pandas as pd

data = Dataset.load_builtin('ml-100k')
train, test = train_test_split(data, test_size=0.2)
algo = myNMF(train)
algo.train(30, True)

i=0 error:232.8120358078912
i=1 error:213.42315809412963
i=2 error:206.41986257969202
i=3 error:202.00706831966613
i=4 error:198.9739669669215
i=5 error:196.81992676427058
i=6 error:195.2111056197175
i=7 error:193.93928952431767
i=8 error:192.89489359182832
i=9 error:192.0309351721562
i=10 error:191.3088473306569
i=11 error:190.6983874250896
i=12 error:190.1712560275384
i=13 error:189.70394419866847
i=14 error:189.2915417633111
i=15 error:188.9258267913843
i=16 error:188.59774599294067
i=17 error:188.29884716694758
i=18 error:188.02385207774915
i=19 error:187.76656340410327
i=20 error:187.52426356251857
i=21 error:187.2982999417575
i=22 error:187.09008358213654
i=23 error:186.89809802616654
i=24 error:186.72302120405826
i=25 error:186.56182289728784
i=26 error:186.4131140370305
i=27 error:186.27590592884624
i=28 error:186.1492841892283
i=29 error:186.0322000997855


The error(Norm of the difference of truths and predictions matrices) seems to converage at around 20+ epochs. Choosing an iteration size like 15 would be significantly faster without much more loss to the accuracy of the model.

In [ ]:
data = Dataset.load_builtin('ml-100k')
train, test = train_test_split(data, test_size=0.2)

for reg in [0.01, 0.02, 0.05, 0.1, 0.25, .5]:
  algo = myNMF(train, 15, reg, reg)
  print(f'\nUsing regularization parameters (lu, li) = {reg}')
  algo.train(15, True)



Using regularization parameters (lu, li) = 0.01
i=0 error:217.1520734515188
i=1 error:193.58816153912576
i=2 error:184.46627790563326
i=3 error:179.01524813872402
i=4 error:175.28102075149508
i=5 error:172.55190093856933
i=6 error:170.45463622894124
i=7 error:168.7702970127201
i=8 error:167.40520044559955
i=9 error:166.28989285319122
i=10 error:165.36219066141513
i=11 error:164.57935907536552
i=12 error:163.91314874804024
i=13 error:163.3379110906859
i=14 error:162.832431459497

Using regularization parameters (lu, li) = 0.02
i=0 error:215.15258004745903
i=1 error:191.48132247258982
i=2 error:182.4105574164401
i=3 error:177.11356670688835
i=4 error:173.59576652343165
i=5 error:171.0768683411867
i=6 error:169.1835549355447
i=7 error:167.72752784285308
i=8 error:166.56951745090197
i=9 error:165.60742827422862
i=10 error:164.78235036942655
i=11 error:164.06262119815358
i=12 error:163.42307820028637
i=13 error:162.8516844563025
i=14 error:162.3392189676657

Using regularization parameters

Convergence speed remains largely unchanged even for much higher lambda values. Error seems to be minimized as **li,lu = 0.05**